## Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using this dataset of 102 flower categories, you can see a few examples below. 

![mpl001](mpl001.png)


The project is broken down into multiple steps:

    Load and preprocess the image dataset
    Train the image classifier on your dataset
    Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import json
from PIL import Image 
import matplotlib.pyplot as plt
import workspace_utils 
from workspace_utils import active_session

# Print statement to verify that imports were successful
print("All modules imported successfully.")

### Load the data

Here you'll use torchvision to load the data (documentation). The data should be included alongside this notebook, otherwise you can download it here.

If you do not find the flowers/ dataset in the current directory, /workspace/home/aipnd-project/, you can download it using the following commands.

!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers

### Data Description

The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225], calculated from the ImageNet images. These values will shift each color channel to be centered at 0 and range from -1 to 1.


In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# Print statements to verify directory paths
print("Training directory path:", train_dir)
print("Validation directory path:", valid_dir)
print("Testing directory path:", test_dir)

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224,0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224,0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224,0.225])
    ])
}

# Idicate that the program is attempting to load the data
print("Attempting to load datasets...")
# TODO: Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test']),
}

class Subset(torch.utils.data.Dataset):
    """
    Subset of a dataset at specified indices
    """
    
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices
        
    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]
    
    def __len__(self):
        return len(self.indices)

# Usage:
num_samples = 100
indices = list(range(num_samples))
small_train_dataset = Subset(image_datasets['train'], indices)

#Create a small dataloader for testing/development
small_dataloader = torch.utils.data.DataLoader(
    small_train_dataset,
    batch_size=64,
    shuffle=True
)

# Confirm that the datasets were loaded successfully by printing their sizes
print("Datasets loaded successfully.")
print(f"Number of training images: {len(image_datasets['train'])}")
print(f"Number of validation images: {len(image_datasets['valid'])}")
print(f"Number of testing images: {len(image_datasets['test'])}")

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=32),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=32),
}    
print(f"Number of images in the small training dataset: {len(small_train_dataset)}")      

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file cat_to_name.json. It's a JSON object which you can read in with the json module. This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.


In [ ]:
import json

# Define the file path
file_path = 'cat_to_name.json'

# Try to open the file
try:
    with open(file_path, 'r') as f:
        cat_to_name = json.load(f)
    print("File loaded successfully.")
    
# Handle the case where the file was not found
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. ")
    
# Handle the case where the file is there, but there is an error in loading its content
except json.JSONDecodeError:
    print(f"Error: There was an error decoding the JSON file '{file_path}'.")
    
# Handle other exceptions
except Exception as e:
    print(f"An unexpected error occured: {e}")

## Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from torchvision.models to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to the rubric for guidance on successfully completing this section. Things you'll need to do:

    Load a pre-trained network (If you need a starting point, the VGG networks work great and are straightforward to use)
    Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
    Train the classifier layers using backpropagation using the pre-trained network to get the features
    Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

### Note for Workspace users:

If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with ls -lh), you should reduce the size of your hidden layers and train again.


In [ ]:
# TODO: Build and train your network
# Load the pre-trained model
chosen_arch = "vgg13" # can be "vgg13" or "vgg16"

if chosen_arch == "vgg13":
    model = models.vgg13(pretrained=True)
elif chosen_arch == "vgg16":
    model = models.vgg16(pretrained=True)
else:
    raise ValueError("Invalid architecture chosen. Select either 'vgg13' or 'vgg16'.")
print(f"{chosen_arch} model loaded successfully.")    

# Print the architecture of the pre-trained model
print("Pre-trained Model Architecture:")
print(model)

# Freeze parameters to prevent backpropagation through them
for param in model.parameters():
    param.requires_grad = False
    
# Define a new classifier
classifier = nn.Sequential(
    nn.Linear(25088, 1024),  
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Dropout(0.3),  
    nn.Linear(256, 102), # 102 is the number of flower categories in this dataset
    nn.LogSoftmax(dim=1)
)

# Replace the existing classifier in the pre-trained model
model.classifier = classifier

# Load the mapping from category label to category name
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
# Define loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Move model to GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #uncomment this when move to gpu
#device = torch.device("cpu") # use this when training in cpu
model.to(device)

import time

print("Start Training on Small Dataset...")

# Training loop on small dataset
small_epochs = 2
steps = 0
running_loss = 0

start_time_small = time.time()

for epoch in range(small_epochs):
    print(f"Starting epoch {epoch+1}/{small_epochs} of Small Dataset Training")
    running_loss = 0
    
    # Use small_dataloader
    for batch, (inputs, labels) in enumerate(small_dataloader):
        print(f"Processing batch {batch+1}/{len(small_dataloader)}")
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        #print("Running zero grad...")
        optimizer.zero_grad()
        #print("Running forward pass...")
        outputs = model(inputs)
        #print("Output batch shape:", outputs.shape) # delete this after I can see training runs successfully
        #print("Computing loss...")
        loss = criterion(outputs, labels)
        #print("Running backward pass...")
        loss.backward()
        #print("Running optimizer step...")
        optimizer.step()
        running_loss += loss.item()
    print(f"Batch {batch+1}/{len(small_dataloader)} processing complete.\n")    
    print(f"Epoch {epoch+1}/{small_epochs} of Small Dataset Training complete.\n")

end_time_small = time.time()    
print("Training on Small Dataset complete")     
print(f"Time elapsed for Small Dataset: {end_time_small - start_time_small:.2f} seconds\n")
    
# Training loop on whole dataset
epochs = 10 
print_every = 50
steps = 0
running_loss = 0

print("Starting Training on Whole Dataset...")
print_interval = 60
start_time = time.time()
print("Time elapsed: ", time.time() - start_time)

last_time = time.time() 
last_status_time = time.time()

#with workspace_utils.active_session():
try:
    for epoch in range(epochs):
        print(f"Starting epoch {epoch+1}/{epochs} of Whole Dataset Training")
        running_loss = 0

        for batch, (inputs, labels) in enumerate(dataloaders['train']):
            #print("Input batch shape:", inputs.shape) # Delete after I know its working
            #print("Labels batch shape:", labels.shape) # Delete after I know its working
            #print(f"Processing batch {batch+1}/{len(dataloaders['train'])}")
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            #print("Output batch shape:", outputs.shape) # Delete this after I know its working
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #Print batch progress
            if steps % print_every == 0:
                valid_loss = 0
                valid_accuracy = 0 
                model.eval()

                with torch.no_grad():
                    for inputs, labels in dataloaders['valid']:
                        #print("Validation input batch shape:", inputs.shape) # Delete after I know its working
                        #print("Validation labels batch shape:", labels.shape) # Delete after I know its working
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        #print("Validation output batch shape:", outputs.shape) #Delete when I know it works
                        batch_loss = criterion(outputs, labels)

                        valid_loss += batch_loss.item()

                        # Calculate accuracy
                        ps = torch.exp(outputs)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        valid_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                # Check the time
                current_time = time.time()
                if current_time - last_time >= print_interval:
                    print(f"...still running after {current_time - last_time:.2f} seconds...")
                    last_time = current_time

                if current_time - last_status_time >= 300:
                    print(
                        f"Status update: Currently on epoch {epoch+1}/{epochs}, "
                        f"batch {batch+1}/{len(dataloaders['train'])}"
                    )                     
                    last_status_time = current_time   

                print(f"Epoch {epoch+1}/{epochs}.."
                    f"Batch {batch+1}/{len(dataloaders['train'])}.. "
                    f"Train loss: {running_loss/print_every:.3f}.. "
                    f"Validation loss: {valid_loss/len(dataloaders['valid']):.3f}.. "
                    f"Validation accuracy: {valid_accuracy/len(dataloaders['valid']):.3f}")

                running_loss = 0
                model.train()  # Return to training mode
            #print(f"Batch {batch+1}/{len(dataloaders['train'])} processing completed.")
        print(f"Epoch {epoch+1}/{epochs} of Whole Dataset completed.\n")                  

    end_time = time.time()        
    print(f"Training on Whole Dataset complete. Time elapsed: {end_time - start_time:.2f} seconds")       

except Exception as e:
    print(f"An error occurred during training: {e}")
    import traceback
    traceback.print_exc()


### Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.


In [ ]:
# TODO: Do validation on the test set
# Testing loop
model.eval()
test_loss = 0
valid_accuracy = 0


with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        test_loss += criterion(outputs, labels).item()

        ps = torch.exp(outputs)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        valid_accuracy += torch.mean(equals.type(torch.FloatTensor))

print(f"Test loss: {test_loss/len(dataloaders['test']):.3f}.."
    f"Test accuracy: {valid_accuracy/len(dataloaders['test']):.3f}")
                  

### Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: image_datasets['train'].class_to_idx. You can attach this to the model as an attribute which makes inference easier later on.

model.class_to_idx = image_datasets['train'].class_to_idx

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, optimizer.state_dict. You'll likely want to use this trained model in the next part of the project, so best to save it now.


In [ ]:
# TODO: Save the checkpoint 
checkpoint = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'classifier': classifier,
    'class_to_idx': image_datasets['train'].class_to_idx
}
torch.save(checkpoint, 'flower_classifier_checkpoint.pth')

# Print a statement to indicate taht the checkpoint has been saved successfuly
print("Checkpoint saved successfully as 'flower_classifier_checkpoint.pth'")

### Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.


In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# Load the checkpoint file
checkpoint = torch.load('flower_classifier_checkpoint.pth')
print("Checkpoint loaded successfully from 'flower_classifier_checkpoint.pth'")

# Rebuild the model by loading the saved state directory from the checkpoint
model.load_state_dict(checkpoint['model'])
print("Model state directory loaded successfully.")

# Load the saved classifier from the checkpoint
model.classifier = checkpoint['classifier']
print("Classifier loaded successfully.")

# Load the class_to_idx attribute
model.class_to_idx = checkpoint["class_to_idx"]
print("class_to_idx loaded successfully.")

# Move the model to the appropriate device (CPU or GPU)
model.to(device)
print(f"Model moved to device: {device}")

# Print a statement if the model rebuilding was successful
print("Model has been rebuilt successfully.")

## Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called predict that takes an image and a model, then returns the top 𝐾

most likely classes along with the probabilities. It should look like

probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']

First you'll need to handle processing the input image such that it can be used in your network.

### Image Preprocessing

You'll want to use PIL to load the image (documentation). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training.

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the thumbnail or resize methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so np_image = np.array(pil_image).

As before, the network expects the images to be normalized in a specific way. For the means, it's [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225]. You'll want to subtract the means from each color channel, then divide by the standard deviation.

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using ndarray.transpose. The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
import os
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    # Load the image using PIL
    pil_image = Image.open(image)
    #print("Image loaded successfully.")
    
    # Resize
    pil_image = pil_image.resize((256, 256))
    #print("Image resized to 256x256.")
    
    # Crop the center
    width, height = pil_image.size
    left = (width - 224) / 2
    top = (height - 224) / 2
    right = (width + 224) /2
    bottom = (height + 224) / 2
    pil_image = pil_image.crop((left, top, right, bottom))
    #print("Center of the image cropped.")
    
    # Convert to numpy array
    np_image = np.array(pil_image) / 255.0
    #print("Image converted to a numpy array and scaled to [0,1].")
    
    # Normalize
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std
    #print("Image normalized.")
    
    # Transpose
    np_image = np_image.transpose((2, 0, 1))
    #print("Image transposed")
    
    return np_image

def process_all_images(root_dir):
    processed_images = []
    
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            #Checkif the file is an image
            if filename.lower().endswith(('.png', '.jpg', '.jpeg','.bmp', '.tiff', '.gif')):
                # Create the full path of the file by joining its directory path and filename
                image_path = os.path.join(dirpath,filename)

                # Process the image using your existing function
                processed_image = process_image(image_path)

                # Append the processed image to the list
                processed_images.append(processed_image)
    
    print("Processing of all images is complete.")
    return processed_images

root_dir = 'flowers/test/'
processed_images = process_all_images(root_dir)
print("Number of images processed:", len(processed_images))

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your process_image function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
        
    # Print the shape of the input image tensor for verification
    #print("Input Image Tensor Shape:", image.shape)
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    if isinstance(image, np.ndarray):
        # If image is a numpy array, no need to convers
        pass
    else:
        # If image is a PyTorch tensor, convert to numpy array
        image = image.numpy()
    
    image = image.transpose((1, 2, 0))
    
    # Print the shape of teh rearranged image array for verification
    #print("Rearranged Image Array Shape:", image.shape)
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    if title:
        ax.set_title(title)
        
    # Print a confirmation that the image preprocessing has been reversed
    #print("Image preprocessing reversed; ready for display.")
    
    
    return ax

# Process all imagees in the 'flowers/test/' directory
root_dir = 'flowers/test/'
processed_images = process_all_images(root_dir)

# Check if there are any processed images
if len(processed_images) > 0:
    # Visualise the first processed image in the test set
    imshow(processed_images[0], title="Flower image")
    plt.show() # Display the image
    print("Cell has completed successfully, and image displayed.")
else:
    print("No images were processed.")

### Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-𝐾
) most probable classes. You'll want to calculate the class probabilities then find the 𝐾

largest values.

To get the top 𝐾

largest values in a tensor use x.topk(k). This method returns both the highest k probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using class_to_idx which hopefully you added to the model or from an ImageFolder you used to load the data (see here). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    
    # Process the image
    processed_image = process_image(image_path)
    
    # Print the shape of the processed image for verification
    #print("Processed Image Shape:", processed_image.shape)
    
    # Convert the numpy array to a PyTorch tensor
    image_tensor = torch.from_numpy(processed_image).type(torch.FloatTensor)
    
    # Add a batch dimension to the image
    image_tensor = image_tensor.unsqueeze(0)
    
    # Move the image tensor to the same device as the model
    image_tensor = image_tensor.to(device)
    
    # Set teh model to evalutaion mode
    model.eval()
    
    # Disable gradient calculations
    with torch.no_grad():
        # Forward pass
        output = model(image_tensor)
        
        # Print the raw output for verification
        print("Model Raw Output:", output)
        
        # Calculate the probabilties adn class indices
        probs, indices = torch.topk(torch.exp(output), topk)
        
        # Convert indices to class labels
        class_to_idx = model.class_to_idx
        idx_to_class = {v: k for k, v in class_to_idx.items()}
        classes = [idx_to_class[idx.item()] for idx in indices[0]]
        
    # Print the final predicted classes and their probabilites for verification
    print("Predicted Classes:", classes)
    print("Prdicted Probabilities:", probs[0].cpu().numpy())
    print("Prediction completed successfully.")
    
        
    return probs[0].cpu().numpy(), classes
    
    

### Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use matplotlib to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

![mpl002](mpl002.png)


You can convert from the class integer encoding to actual flower names with the cat_to_name.json file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the imshow function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

# Directory path for test images
root_dir = 'flowers/test/'

# Iterate through each sub-folder (corresponding to a flower class)
for sub_folder in os.listdir(root_dir):
    sub_folder_path = os.path.join(root_dir, sub_folder)
    
    # Check if the path is indeed a directory (to ensure it's not a stray file)
    if os.path.isdir(sub_folder_path):
        # Get the list of images in the sub-folder
        images = [img for img in os.listdir(sub_folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg','.bmp', '.tiff', '.gif'))]

        # If there are images in the sub-folder
        if images:
            # choose an image for demonstration purposes
            image_path = os.path.join(sub_folder_path, images[0])

            # Predict the top 5 classes and their probabilities
            probs, classes = predict(image_path, model, topk=5)

            # Convert class indices to class names using cat_to_name dictioary
            class_names = [cat_to_name[cls] for cls in classes]

            # Print the predicitons for verification
            print(f"Predictions for image: {image_path}")
            print('Top Classes:', class_names)
            print('Top Probabilities:', probs)

            # Load and process the image for display
            processed_image = process_image(image_path)
            imshow(processed_image, title=class_names[0])

            # Plot the bar chart for the top 5 classes and their probabilites
            plt.figure(figsize=(6,10))
            plt.subplot(2,1,2)
            plt.barh(class_names, probs)
            plt.xlabel('Probabilities')
            plt.title('Top 5 Predicted Flower Classes')
            plt.gca().invert_yaxis() # Invert y-axis to show the highest probability

            plt.tight_layout()
            plt.show()

print("All predictions completed successfully.")

## Reminder for Workspace users

If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again.

We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.


In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace